Installing libraries

In [1]:
!pip install openai
!pip install textblob
!pip install python-dotenv

Importing libraries

In [2]:
import os
import openai
import time
import logging
from openai import AzureOpenAI

In [43]:
import os
import openai
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from dotenv import load_dotenv
from operator import itemgetter
from textblob import TextBlob
import json
import requests
import copy

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SXM0326\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\SXM0326\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SXM0326\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SXM0326\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\SXM0326\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Initialization

In [4]:
client = AzureOpenAI(  
                azure_endpoint="https://sbd-genai-apollo-useast2-sbx-openai.openai.azure.com/",  
                api_key="1630bbf79d9e44dfb1417053c38cc8ec",    
                api_version="2024-02-15-preview")

In [5]:
os.chdir('c:\\Users\\SXM0326\\Downloads\\Extract_files')

In [6]:
VENDOR_STOPWORDS=['BIZ',
                  'BV',
                  'CO',
                  'COMP',
                  'COMPANY',
                  'COMPANIES',
                  'CORP',
                  'CORP',
                  'CORPORATE',
                  'STORE',
                  'STORES',
                  'RDC',
                  'BDC',
                  '<DI>',
                  'DFC',
                  'OF',
                  'THE',
                  '(DI)',
                  '<DI>',
                  '<DI>',
                  'DIST',
                  'DISTRIBUTION',
                  '.COM',
                  'CORPORATION',
                  'CONFIDENTIAL',
                  'SERVICE',
                  'SERVICES',
                  'LLC',
                  'LTD',
                  'LLP',
                  'INC',
                  'INCORP',
                  'INCORPORAT',
                  'INCORPORATE',
                  'INCORPORATED',
                  'INCORPORATION',
                  'INTERNATIONAL',
                  'INTL',
                  'INTNL',
                  'LIMITED',
                  'LTD',
                  'LLC',
                  'LTD',
                  'LLP',
                  'MACHINES',
                  'PVT',
                  'PTE',
                  'PRIVATE',
                  'HDWE',
                  'HARDWARE',
                  'HARDWARES',
                  'DROPSHIP',
                  'DROPSHIPPING',
                  'DROPSHIPPER',
                  'DROPSHIPPER',
                  'LOC',
                  'LOCATION',
                  'PT',
                  'POINT',
                  'DTS',
                  'INDUSTRITIES',
                  'INDUSTRIES',
                  'INDUSTRIAL',
                  'INDUSTRY',
                  'DEDC',
                  'VENDOR',
                  'VENDORS',
                  'IVSC',
                  'STATE',
                  'FULFILLMENT',
                  'FULFILLMENTS',
                  'SERVICE',
                  'SERVICES',
                  'DEDC',
                  'TOOL',
                  'TOOLS',
                  'REPAIR',
                  'DOTCOM'
                  ]

Read File

In [7]:
df = pd.read_csv("na_cust_names_1.csv")
df['XFRM_CUST_NAME'] = df['XFRM_CUST_NAME'].str.upper()
df['XFRM_CUST_NAME'] = df['XFRM_CUST_NAME'].apply(lambda x: x.replace('<DI>',''))
df['XFRM_CUST_NAME'] = df['XFRM_CUST_NAME'].apply(lambda x: x.replace('(DI)',''))
df['XFRM_CUST_NAME'] = df['XFRM_CUST_NAME'].apply(lambda x: ' '.join([word for word in x.split() if word not in (VENDOR_STOPWORDS)]))
df.shape

(1517517, 5)

EDA

In [8]:
df.head()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD
0,SAPC11,0004864903,CHAPPEL LANA,L6Y 6A8,CA
1,SAPC11,0004068784,"KELLEY, JUDY",92821,US
2,SAPC11,0001622054,VALLN IRLN POS ONLY OH WAYNE NO ORDERS POS ONLY,44691,US
3,SAPC11,0009158037,KASSANDRA CHRISTENSEN,58723,US
4,SAPC11,0001160480,DGI POS ONLY ID BANNOCK NO ORDERS POS ONLY,83201,US


In [9]:
df.tail()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD
1517512,SAPC11,0004255544,SHIRLEY HIATT,27030,US
1517513,SAPC11,0009124789,ORANGE COUNTY FIRE AND RESCUE,32811,US
1517514,SAPC11,0004862932,ROMAN VRANZEWSKI,H1S 2X6,CA
1517515,SAPC11,0009083400,"HAWES, WARREN",85739,US
1517516,SAPC11,0004135153,R AND W CABINETRY,23462,US


In [10]:
len(df[df['XFRM_CUST_NAME'].str.contains('LOWES')]), len(df[df['XFRM_CUST_NAME'].str.contains('WALMART')]), len(df[df['XFRM_CUST_NAME'].str.contains('HOME DEPOT')]), len(df[df['XFRM_CUST_NAME'].str.contains('AMAZON')]), len(df[df['XFRM_CUST_NAME'].str.contains('AMZN')])

(13395, 22594, 12908, 3420, 0)

Model Instruction

In [11]:
delimiter = '#'
MODEL = "sbd-gpt-4-turbo-128k"
system_message = f"""
You will be provided with customer names, delimited by {delimiter}. \
The name can either be a person's name or retail company name or blank or empty string.\
if the name is a retail company name, it can be nestled in the string.\
If the name is a person's name, output the name as is. \
If the name is a retail company name, extract the company name only without any number \
or special character or place name or state name or country name or the direction name or any \
abbreviated names or any included acquired company names and output the company name.\
If the name is blank or empty string, then output 'No name found.'
Output has to strictly follow the instruction.
"""

In [12]:
temp_df = df[df['XFRM_CUST_NAME'].str.contains('LOWES')]
temp_df

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD
677,SAPC11,0002822247,LOWES 2247,15102,US
880,SAPC11,0003409092,ANNA ORBEN FIELD OPERATIONS MANAGER LOWES,28037,US
899,SAPC11,0002822591,LOWES 2591,37620,US
1289,SAPC11,0005602776,LOWES 3066,N7S 6M8,CA
1593,SAPC11,0002822613,LOWES 2613,16830,US
...,...,...,...,...,...
1515612,SAPC11,0002822284,LOWES 2284,11234,US
1515614,SAPC11,0002821181,LOWES 1181,36066,US
1516683,SAPC11,0001909614,LOWES 3708 3708,K1B 3K5,CA
1516760,SAPC11,0002821099,LOWES 1099,80922,US


In [13]:
def get_completion_from_messages(messages, model=MODEL, temperature=0, max_tokens=4096):
    response = client.chat.completions.create(model=MODEL,
                    messages= messages,
                    temperature=0,
                    max_tokens=max_tokens,
                    top_p=0.95,
                    frequency_penalty=0,
                    presence_penalty=0,
                    stop=None)
    return response.choices[0].message.content  

In [14]:
len(temp_df)

13395

In [15]:
refined_name_list = []
start_idx = 0
for curr_idx, name in enumerate(temp_df['XFRM_CUST_NAME'].values.tolist()):
    user_message_1 = f"""Extract the company name for {name} """
    messages =  [
    {'role':'system',
     'content': system_message},
    {'role':'user',
     'content': f"{delimiter}{user_message_1}{delimiter}"},
    ]
    response = get_completion_from_messages(messages)
    refined_name_list.append((name, response))
    if curr_idx % 100 == 0 and curr_idx > 0:
        work_df = copy.deepcopy(temp_df.iloc[start_idx : curr_idx + 1])
        work_df['REFINED_XFRM_CUST_NAME'] = [e[1] for e in refined_name_list]
        work_df['REFINED_XFRM_CUST_NAME'] = work_df['REFINED_XFRM_CUST_NAME'].apply(lambda x : x.upper().replace("'", ""))
        work_df.sort_values(by = ['XFRM_PSTL_CD_1'], inplace = True)
        print(curr_idx)
        start_idx = curr_idx + 1
        work_df.to_csv(f"temp_file_{curr_idx}.csv", index = False)
        refined_name_list = []

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300


In [16]:
refined_name_list[:20]

[('LOWES 2752', 'LOWES'),
 ('LOWES 677', 'LOWES'),
 ('LOWES SUPERMARKET 029', 'LOWES'),
 ('LOWES CRFT', 'LOWES'),
 ('LOWES 2878', 'LOWES'),
 ('LOWES (RONA) 83097 83097', 'LOWES'),
 ('LOWES SUPERMARKET 65', 'LOWES'),
 ('LOWES 1648', 'LOWES'),
 ('LOWES #0332', 'LOWES'),
 ('LOWES 3313 3313', 'LOWES'),
 ('LOWES', 'LOWES'),
 ('LOWES 965 S JAX', 'LOWES'),
 ('CTN DIR LOWES 1415 1415', 'LOWES'),
 ('LOWES 990', 'LOWES'),
 ('LOWES 1043', 'LOWES'),
 ('LOWES (RONA) 88009 88009', 'LOWES'),
 ('LOWES (RONA) 83201 83201', 'LOWES'),
 ('LOWES 547', 'LOWES'),
 ('LOWES 1567', 'LOWES'),
 ('LOWES #0301', 'LOWES')]

In [17]:
import glob
import fileinput

In [18]:
with open('temp.csv', 'w') as fout, fileinput.input(glob.glob(os.path.join(os.getcwd(), 'temp_file*.csv'))) as fin:
    for line in fin:
        fout.write(line)

In [30]:
df_lowes = pd.read_csv("temp.csv")
df_lowes[df_lowes['SRC_SYS_KEY'] == 'SRC_SYS_KEY']
df_lowes = df_lowes[df_lowes['SRC_SYS_KEY'] != 'SRC_SYS_KEY']
df_lowes.columns = ['SRC_SYS_KEY',	
                    'CUST_NBR',	
                    'XFRM_CUST_NAME',	
                    'XFRM_PSTL_CD_1',	
                    'XFRM_CNTRY_CD',	
                    'REFINED_XFRM_CUST_NAME'
                ]
df_lowes.shape

(13301, 6)

In [31]:
df_lowes.head()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
0,EXCEL_QADDB,SOS02417,"LOWES SCOTT TOWNSHIP, PA",15106,US,LOWES
1,EXCEL_QADDB,SOS02869,"LOWES WHITEVILLE, NC",28472,US,LOWES
2,EXCEL_QADDB,SOS01725,"LOWES NE AUSTIN, TX",78753,US,LOWES
3,EXCEL_QADDB,SOS03348,LOWES CENTRAL DELIVERY TERMINAL,29492,US,LOWES
4,EXCEL_QADDB,LWS-2786,"LOWES S SAN ANTONIO, TX",78224,US,LOWES


In [32]:
df_lowes.tail()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
13428,EXCEL_QADDB,SOS00004,"LOWES RAINIER, WA",98144,US,LOWES
13429,EXCEL_QADDB,SOS02927,LOWES HOME IMPROVEMENT,12534,US,Lowe's
13430,EXCEL_QADDB,LWS-2524,"LOWES GREENVILLE, OH",45331,US,LOWES
13431,EXCEL_QADDB,SOS01721,"LOWES SOUTHAVEN, MS",38671,US,LOWES
13432,EXCEL_QADDB,SOS00424,"LOWES SALISBURY, MD",21801,US,LOWES


In [47]:
df_lowes['REFINED_XFRM_CUST_NAME'] = df_lowes['REFINED_XFRM_CUST_NAME'].apply(lambda x : x.upper().replace("'", "").replace("\r", " ").replace("\n", " ").replace("#NO NAME FOUND.", "").strip())
df_lowes.sort_values(by = ['XFRM_PSTL_CD_1'], inplace=True)
df_lowes.head()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
4547,INFORXA,3308,#1916 LOWES,01035,US,LOWES
10725,INFORXA,12840,#1916 LOWES HADLEY MA,01035,US,LOWES
11139,INFORXA,3308,#1916 LOWES HADLEY MA,01035,US,LOWES
9059,INFORXA,3308,#1916 LOWES HADLEY,01035,US,LOWES
9175,EXCEL_QADDB,SOS01916,"LOWES HADLEY, MA",01035,US,LOWES


In [48]:
df_lowes.groupby(['REFINED_XFRM_CUST_NAME'])['REFINED_XFRM_CUST_NAME'].value_counts()

REFINED_XFRM_CUST_NAME
ACE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     5
ACE HARDWARE  LOWES                                                                                                                                                                                                                                                                                                                                                                                                                                                                     1
ACE HARDWARE  NO NAME FOUND. 

In [49]:
df_lowes['REFINED_XFRM_CUST_NAME'] = np.where(df_lowes['REFINED_XFRM_CUST_NAME'].str.contains('LOWES BALLWIN'), 'LOWES', df_lowes['REFINED_XFRM_CUST_NAME'])

In [50]:
df_lowes['REFINED_XFRM_CUST_NAME'].drop_duplicates().to_list()

['LOWES',
 'LOWES#LOWES',
 'LOWES MAERSK CHESAPEAKE TRANSLOAD 3507    COMPANY NAME: LOWES',
 'LOWES MAERSK CHESAPEAKE TRANSLOAD',
 'LOWES TRUE VALUE',
 'LOWES FOOD',
 'LOWES # FAO NATHAN',
 'NO NAME FOUND.',
 'LOWES COMPANIES',
 'LOWES # NO NAME FOUND.',
 'LOWES#',
 'LOWES JOME IMPROVEMENT',
 'LOWES # NO NAME FOUND. # NO NAME FOUND.',
 'MTD PRODUCTS',
 'LOWES PRO SPLY',
 'ACE',
 'LOWES FLEXE',
 'LOWES#HUFFY',
 'LOWES MARKET',
 'LOWES FAMILY CENTER ACE',
 'LOWES  RAYMOND',
 'ACE HARDWARE  NO NAME FOUND.',
 'LOWES FAMILY CENTER',
 'ACE HARDWARELOWES ACE    THE INPUT SEEMS TO CONTAIN TWO RETAIL COMPANY NAMES CONCATENATED TOGETHER WITHOUT PROPER DELIMITERS. HOWEVER, BASED ON THE INSTRUCTIONS, WE NEED TO EXTRACT THE COMPANY NAME WITHOUT ANY NUMBERS, SPECIAL CHARACTERS, PLACE NAMES, STATE NAMES, COUNTRY NAMES, DIRECTION NAMES, ABBREVIATED NAMES, OR INCLUDED ACQUIRED COMPANY NAMES.    THE TWO COMPANY NAMES ARE "ACE HARDWARE" AND "LOWES." THEREFORE, THE OUTPUT SHOULD BE:    ACE HARDWARE  LOWES

In [51]:
df_lowes['REFINED_XFRM_CUST_NAME'] = np.where(df_lowes['REFINED_XFRM_CUST_NAME'].str.contains('LOWES'), 'LOWES', df_lowes['REFINED_XFRM_CUST_NAME'])

In [53]:
df_lowes['REFINED_XFRM_CUST_NAME'] = np.where(df_lowes['REFINED_XFRM_CUST_NAME'].str.contains('RONA'), 'LOWES', df_lowes['REFINED_XFRM_CUST_NAME'])

In [54]:
df_lowes['REFINED_XFRM_CUST_NAME'].drop_duplicates().to_list()

['LOWES',
 'NO NAME FOUND.',
 'MTD PRODUCTS',
 'ACE',
 'ACE HARDWARE  NO NAME FOUND.']

In [56]:
df_lowes[df_lowes['REFINED_XFRM_CUST_NAME'] == 'NO NAME FOUND.']

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
12270,SAPC11,0001016828,LOWES POS ACCOUNT ONLY,28656,US,NO NAME FOUND.
5475,SAPC11,0004074221,DEVIN SOLOWES,35006,US,NO NAME FOUND.
1121,SAPC11,0004185243,"CLOWES, GEORGE",72023,US,NO NAME FOUND.
6115,INFORXA,12840,"#2479 LOWES BRIGHTON,",80601,US,NO NAME FOUND.
4829,SAPC11,0004721481,LOWES KATHRYN DEGROOT,L2H 0G5,CA,NO NAME FOUND.
1192,SAPC11,0004849282,"BREWER CLOWES,MICHEAL",V5M 3Y7,CA,NO NAME FOUND.


In [57]:
df_lowes = df_lowes[~df_lowes['XFRM_CUST_NAME'].isin(['DEVIN SOLOWES', 'CLOWES, GEORGE', 'BREWER CLOWES,MICHEAL'])]
df_lowes[df_lowes['REFINED_XFRM_CUST_NAME'] == 'NO NAME FOUND.']

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
12270,SAPC11,0001016828,LOWES POS ACCOUNT ONLY,28656,US,NO NAME FOUND.
6115,INFORXA,12840,"#2479 LOWES BRIGHTON,",80601,US,NO NAME FOUND.
4829,SAPC11,0004721481,LOWES KATHRYN DEGROOT,L2H 0G5,CA,NO NAME FOUND.


In [58]:
df_lowes['REFINED_XFRM_CUST_NAME'] = np.where(df_lowes['XFRM_CUST_NAME'].str.contains('LOWES'), 'LOWES', df_lowes['REFINED_XFRM_CUST_NAME'])

In [59]:
df_lowes['REFINED_XFRM_CUST_NAME'].drop_duplicates().to_list()

['LOWES']

In [70]:
df_lowes[df_lowes['XFRM_PSTL_CD_1'] == 'V9B 2P3']

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
12022,SAPC11,0001909573,LOWES 3702 SWK CLOSED PER MKTG 05032021,V9B 2P3,CA,LOWES
12741,SAPC11,0001204940,LOWES 3702 3702,V9B 2P3,CA,LOWES
3697,SAPC11,0001909575,LOWES 3702 3702,V9B 2P3,CA,LOWES
1804,SAPC11,0004831711,LOWES HOME IMPROVEMENT 3702,V9B 2P3,CA,LOWES
11872,SAPC11,0001205326,LOWES 3702 3702,V9B 2P3,CA,LOWES
6925,SAPC11,0001213291,LOWES (RONA) 83702 83702,V9B 2P3,CA,LOWES
4273,SAPC11,0001909574,LOWES 3702 3702,V9B 2P3,CA,LOWES
5416,SAPC11,0005604196,LOWES 3702 3702,V9B 2P3,CA,LOWES
12849,SAPC11,0001213292,LOWES (RONA) 83702 83702,V9B 2P3,CA,LOWES
3682,SAPC11,0005604333,LOWES (RONA) 83702 83702,V9B 2P3,CA,LOWES


In [69]:
df_lowes.groupby(['XFRM_PSTL_CD_1', 'XFRM_CNTRY_CD', 'REFINED_XFRM_CUST_NAME']).count()

,,,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME
XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME,,,
01035,US,LOWES,6,6,6
01082,US,LOWES,6,6,6
01129,US,LOWES,5,5,5
01453,US,LOWES,6,6,6
01581,US,LOWES,6,6,6
...,...,...,...,...,...
V9B 2P3,CA,LOWES,11,11,11
V9B 2P3,CA,LOWES,4,4,4
V9T 4K6,CA,LOWES,11,11,11


In [71]:
df_lowes[df_lowes['XFRM_PSTL_CD_1'] == '01035']

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
4547,INFORXA,3308,#1916 LOWES,01035,US,LOWES
10725,INFORXA,12840,#1916 LOWES HADLEY MA,01035,US,LOWES
11139,INFORXA,3308,#1916 LOWES HADLEY MA,01035,US,LOWES
9059,INFORXA,3308,#1916 LOWES HADLEY,01035,US,LOWES
9175,EXCEL_QADDB,SOS01916,"LOWES HADLEY, MA",01035,US,LOWES
5376,SAPC11,0002821916,LOWES 1916,01035,US,LOWES


In [72]:
df_lowes['CUST_GOLDEN_REC_KEY'] = df_lowes[['XFRM_CNTRY_CD', 'XFRM_PSTL_CD_1', 'REFINED_XFRM_CUST_NAME']].apply(lambda x: '~'.join(x), axis=1)

In [77]:
df_lowes.sort_values(by = ['CUST_GOLDEN_REC_KEY'], inplace=True)
df_lowes.reset_index(drop = True, inplace = True)

In [79]:
df_lowes.head()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME,CUST_GOLDEN_REC_KEY
0,INFORXA,16257,RONALOWES,J4B 8H7,CA,LOWES,CA~J4B 8H7 ~LOWES
1,INFORXA,7991,LOWES #3708,K1B 3K5,CA,LOWES,CA~K1B 3K5 ~LOWES
2,INFORXA,16257,LOWES #3708 GLOUCESTER,K1B 3K5,CA,LOWES,CA~K1B 3K5 ~LOWES
3,SAPC11,0001204973,LOWES 3708 3708,K1B 3K5,CA,LOWES,CA~K1B 3K5~LOWES
4,SAPC11,0001930716,LOWES (RONA) 83708 83708,K1B 3K5,CA,LOWES,CA~K1B 3K5~LOWES


In [78]:
df_lowes.tail()

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME,CUST_GOLDEN_REC_KEY
13293,SAPC11,0002821985,LOWES 1985 LOWES FAIRBANKS,99701,US,LOWES,US~99701~LOWES
13294,INFORXA,3308,#1985 LOWES FAIRBANKS,99701,US,LOWES,US~99701~LOWES
13295,INFORXA,12840,"#1985 LOWES FAIRBANKS, AK",99701,US,LOWES,US~99701~LOWES
13296,INFORXA,3308,"#1985 LOWES FAIRBANKS, AK",99701,US,LOWES,US~99701~LOWES
13297,SAPC11,0002820964,LOWES IMPORT CONSOL PROG,^^,US,LOWES,US~^^~LOWES


In [80]:
df_lowes['CUST_GOLDEN_REC_KEY'].value_counts()

CUST_GOLDEN_REC_KEY
US~28659~LOWES    58
US~21286~LOWES    54
US~28697~LOWES    40
US~31601~LOWES    37
US~45840~LOWES    36
                  ..
US~71203~LOWES     1
US~70520~LOWES     1
US~64504~LOWES     1
US~63780~LOWES     1
US~^^~LOWES        1
Name: count, Length: 2321, dtype: int64

In [82]:
len(df_lowes['CUST_GOLDEN_REC_KEY'].drop_duplicates().values.tolist()), len(df_lowes['CUST_GOLDEN_REC_KEY'].values.tolist())

(2321, 13298)

In [55]:
df_lowes[df_lowes['REFINED_XFRM_CUST_NAME'] == 'ACE']

,SRC_SYS_KEY,CUST_NBR,XFRM_CUST_NAME,XFRM_PSTL_CD_1,XFRM_CNTRY_CD,REFINED_XFRM_CUST_NAME
5032,SAPC11,0002666254,MARLOWES ACE,60140,US,ACE
8906,INFORXA,1092,MARLOWES ACE,60140,US,ACE
5329,SAPC11,0002543408,LOWES ACE 14067,78016,US,ACE
12656,SAPC11,0002530323,LOWES ACE 148,78017,US,ACE
5210,SAPC11,0002509695,LOWES ACE,79339,US,ACE
